In [0]:
# Read in json data using spark df reader
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [0]:
# Define schema
name_schema = StructType(fields=[StructField("forename", StringType(), True),
                                 StructField("surname", StringType(), True)
  
])

drivers_schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                                    StructField("driverRef", StringType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("code", StringType(), True),
                                    StructField("name", name_schema),
                                    StructField("dob", DateType(), True),
                                    StructField("nationality", StringType(), True),
                                    StructField("url", StringType(), True)  
])

In [0]:
# Read data into df
drivers_df = spark.read \
.schema(drivers_schema) \
.json("/mnt/jdoformula1dl/raw/drivers.json")

In [0]:
# Rename columns for better clarity and add new columns - ingestion date 
from pyspark.sql.functions import current_timestamp, col, concat, lit
drivers_with_columns_df = drivers_df.withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("driverRef", "driver_ref") \
                                    .withColumn("ingestion_date", current_timestamp()) \
                                    .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname")))

In [0]:
# Drop unwanted columns
drivers_final_df = drivers_with_columns_df.drop(col("url"))

In [0]:
# Write output as parquet to processed container
drivers_final_df.write.mode("overwrite").parquet("/mnt/jdoformula1dl/processed/drivers")    